In [46]:
import pandas as pd
import numpy as np
import glob
from sklearn.preprocessing import LabelEncoder

In [47]:
all_files = glob.glob("../data/annotations/*.csv")
dfs = []
for file in all_files:
    df = pd.read_csv(file)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

In [48]:
df

,frame_number,window_size,mean_distance,median_player1_x,median_player1_y,median_player2_x,median_player2_y,video_name,state
0,49,50,3.233949,5.147697,6.179893,2.139223,7.666041,video-2,start
1,58,50,3.153754,4.825581,6.225677,2.097130,7.692301,video-2,active
2,99,50,2.634458,3.981588,6.249024,1.852244,7.937177,video-2,active
3,149,50,1.591595,3.297013,7.048002,2.522338,8.316031,video-2,active
4,199,50,1.933582,3.113780,6.577660,3.157080,8.355049,video-2,active
...,...,...,...,...,...,...,...,...,...
4592,35949,50,2.751492,3.842180,6.122868,2.219147,4.042270,video-5,end
4593,35999,50,2.519606,3.287379,6.838716,2.075434,4.536124,video-5,end
4594,36049,50,2.512097,2.712794,7.346559,2.225436,4.878631,video-5,end
4595,36099,50,2.378643,2.061877,7.491796,2.571463,5.191120,video-5,end


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4597 entries, 0 to 4596
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   frame_number      4597 non-null   int64  
 1   window_size       4597 non-null   int64  
 2   mean_distance     4597 non-null   float64
 3   median_player1_x  4597 non-null   float64
 4   median_player1_y  4597 non-null   float64
 5   median_player2_x  4597 non-null   float64
 6   median_player2_y  4597 non-null   float64
 7   video_name        4597 non-null   object 
 8   state             4597 non-null   object 
dtypes: float64(5), int64(2), object(2)
memory usage: 323.4+ KB


### Distance-based features


In [50]:
# Lagged distance features
df["distance_lag_1"] = df.groupby("video_name")["mean_distance"].shift(1)
df["distance_lag_2"] = df.groupby("video_name")["mean_distance"].shift(2)
df["distance_lag_3"] = df.groupby("video_name")["mean_distance"].shift(3)

# Distance changes/deltas
df["distance_change"] = df.groupby("video_name")["mean_distance"].diff()
df["distance_acceleration"] = df.groupby("video_name")["distance_change"].diff()

# Rolling statistics (3-5 frame windows)
df["distance_rolling_mean"] = (
    df.groupby("video_name")["mean_distance"]
    .rolling(3)
    .mean()
    .reset_index(0, drop=True)
)
df["distance_rolling_std"] = (
    df.groupby("video_name")["mean_distance"].rolling(3).std().reset_index(0, drop=True)
)
df["distance_rolling_min"] = (
    df.groupby("video_name")["mean_distance"].rolling(3).min().reset_index(0, drop=True)
)
df["distance_rolling_max"] = (
    df.groupby("video_name")["mean_distance"].rolling(3).max().reset_index(0, drop=True)
)

### Position-based features


In [51]:
# Player movement features
df["player1_movement"] = np.sqrt(
    df.groupby("video_name")["median_player1_x"].diff() ** 2
    + df.groupby("video_name")["median_player1_y"].diff() ** 2
)
df["player2_movement"] = np.sqrt(
    df.groupby("video_name")["median_player2_x"].diff() ** 2
    + df.groupby("video_name")["median_player2_y"].diff() ** 2
)

# Court position features (relative to court center/service lines)
df["player1_court_side"] = (df["median_player1_x"] > 3.2).astype(
    int
)  # Your court_center_x
df["player2_court_side"] = (df["median_player2_x"] > 3.2).astype(int)
df["players_same_side"] = (df["player1_court_side"] == df["player2_court_side"]).astype(
    int
)

# Distance from service line
df["player1_from_service_line"] = df["median_player1_y"] - 5.44  # Your service_line_y
df["player2_from_service_line"] = df["median_player2_y"] - 5.44

### Temporal features


In [52]:
# Previous state (very powerful feature)
df["prev_state"] = df.groupby("video_name")["state"].shift(1)

# State duration (how long in current state)
df["state_duration"] = df.groupby(["video_name", "state"]).cumcount() + 1

# Time since last state change
df["frames_since_state_change"] = (
    df.groupby("video_name")
    .apply(lambda x: (x["state"] != x["state"].shift(1)).cumsum(), include_groups=False)
    .reset_index(0, drop=True)
)

In [53]:
df

,frame_number,window_size,mean_distance,median_player1_x,median_player1_y,median_player2_x,median_player2_y,video_name,state,distance_lag_1,...,player1_movement,player2_movement,player1_court_side,player2_court_side,players_same_side,player1_from_service_line,player2_from_service_line,prev_state,state_duration,frames_since_state_change
0,49,50,3.233949,5.147697,6.179893,2.139223,7.666041,video-2,start,NaN,...,NaN,NaN,1,0,0,0.739893,2.226041,NaN,1,1
1,58,50,3.153754,4.825581,6.225677,2.097130,7.692301,video-2,active,3.233949,...,0.325354,0.049613,1,0,0,0.785677,2.252301,start,1,2
2,99,50,2.634458,3.981588,6.249024,1.852244,7.937177,video-2,active,3.153754,...,0.844316,0.346314,1,0,0,0.809024,2.497177,active,2,2
3,149,50,1.591595,3.297013,7.048002,2.522338,8.316031,video-2,active,2.634458,...,1.052145,0.769777,1,0,0,1.608002,2.876031,active,3,2
4,199,50,1.933582,3.113780,6.577660,3.157080,8.355049,video-2,active,1.591595,...,0.504773,0.635940,0,0,1,1.137660,2.915049,active,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4592,35949,50,2.751492,3.842180,6.122868,2.219147,4.042270,video-5,end,2.766094,...,1.032533,1.081008,1,0,0,0.682868,-1.397730,end,296,79
4593,35999,50,2.519606,3.287379,6.838716,2.075434,4.536124,video-5,end,2.751492,...,0.905673,0.514340,1,0,0,1.398716,-0.903876,end,297,79
4594,36049,50,2.512097,2.712794,7.346559,2.225436,4.878631,video-5,end,2.519606,...,0.766846,0.373914,0,0,1,1.906559,-0.561369,end,298,79
4595,36099,50,2.378643,2.061877,7.491796,2.571463,5.191120,video-5,end,2.512097,...,0.666923,0.466244,0,0,1,2.051796,-0.248880,end,299,79


In [54]:
df.to_csv("../data/annotations/combined_annotations.csv", index=False)